In [1]:
from os.path import join as opj
import glob
import re
from glm_utils import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [4]:
confounds_list = [
    "trans_x", "trans_y", "trans_z",
    "rot_x", "rot_y", "rot_z",
    "framewise_displacement",
    'a_comp_cor_00', 'a_comp_cor_01','a_comp_cor_02','a_comp_cor_03','a_comp_cor_04','a_comp_cor_05', 
    'csf']
repetition_time = 2.
fd_thresh = 0.5
std_dvars_thresh = 1.5
num_TRs = 168
# Onset time offset (adjust for slice timing correction)
# see https://afni.nimh.nih.gov/pub/dist/doc/program_help/3dTshift.html
# and fMRIPrep implementation https://github.com/nipreps/fmriprep/blob/7f68f590d3abfb7800067a88d74c4f18ac512c29/fmriprep/workflows/bold/stc.py#L100
# tzero = np.round(first + frac * (last - first), 3)
# first and last timing could be found in SliceTiming field in raw data json file
stim_times_subtract = np.round(0 + 0.5 * (1.9 - 0), 3)

derivative_dir = '/projects/kuhl_lab/wanjiag/GLACIER/derivatives/'
fmriprep_dir = opj(derivative_dir, 'fmriprep/')
behav_dir = opj(derivative_dir, 'behavior/')

f_list = [x for x in glob.glob(os.path.join(fmriprep_dir, '*sub-GLACIER*/'))]
subs = list(map(lambda f: f[len(os.path.commonpath(f_list))+1:-1], f_list))
subs.sort()
print(subs)

['sub-GLACIER01', 'sub-GLACIER02', 'sub-GLACIER03', 'sub-GLACIER04', 'sub-GLACIER06', 'sub-GLACIER07', 'sub-GLACIER08', 'sub-GLACIER10', 'sub-GLACIER11', 'sub-GLACIER12', 'sub-GLACIER13', 'sub-GLACIER14', 'sub-GLACIER16', 'sub-GLACIER18']


In [5]:
sub = subs[2]
sub

'sub-GLACIER03'

In [6]:
sub_id = re.findall(r'\d', sub)
sub_id = "".join(list(map(str, sub_id)))

behav_file_list =  [x for x in glob.glob(opj(behav_dir, f'sub{sub_id}', f'sub{sub_id}_exposure*_behav_*.csv'))]
behav_file_list.sort()
timing_file_list = [x for x in glob.glob(opj(behav_dir, f'sub{sub_id}', f'sub{sub_id}_exposure*_timing_*.csv'))]
timing_file_list.sort()

bold_dir=opj(fmriprep_dir, f'{sub}/func/')
out_dir = opj(derivative_dir, f'glm/{sub}')

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

tmp_dir = opj(out_dir, 'temporary_files')
if not os.path.isdir(tmp_dir):
    os.makedirs(tmp_dir)
    
confound_dir = opj(out_dir, 'confound_files')
if not os.path.isdir(confound_dir):
    os.makedirs(confound_dir)

xmat_dir = opj(out_dir, 'xmat_files')
if not os.path.isdir(xmat_dir):
    os.makedirs(xmat_dir)
    
bucket_dir = opj(out_dir, 'bucket_files')
if not os.path.isdir(bucket_dir):
    os.makedirs(bucket_dir)
    
#trl_mod_dir = Path(tmp_dir).joinpath("bucket_files")
#trl_mod_dir.mkdir(exist_ok=True, parents=True)

func_file_list = [x for x in glob.glob(opj(bold_dir, f'{sub}_task-glacier_run-*_space-T1w_desc-preproc_bold.nii.gz'))] 
func_file_list.sort()

# making general mask
mask_file_list = [x for x in glob.glob(opj(bold_dir, f'{sub}_task-glacier_run-*_space-T1w_desc-brain_mask.nii.gz'))]
mask_file_list.sort()

mask_intersect = opj(tmp_dir, f'{sub}_space-T1w_desc-brain_intersect_mask.nii.gz')
if not os.path.exists(mask_intersect): 
    intersect_masks(mask_file_list, mask_intersect)

++ processing 1 input dataset(s), NN=2...
++ padding all datasets by 0 (for dilations)
++ have 8 volumes of input to combine
++ frac 1 over 8 volumes gives min count 8
++ voxel limits: 12540 clipped, 268452 survived, 375336 were zero
++ writing result sub-GLACIER03_space-T1w_desc-brain_intersect_mask.nii.gz...
++ Output dataset /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/temporary_files/sub-GLACIER03_space-T1w_desc-brain_intersect_mask.nii.gz



In [7]:
# fmriprep confound files
confound_file_list = [x for x in glob.glob(opj(bold_dir, f'{sub}_task-glacier_run-*_desc-confounds_timeseries.tsv'))]
confound_file_list.sort()

confounds = []
for f in confound_file_list:
    confounds_run = pd.read_csv(f, sep="\t")

    # add useful information
    confounds_run["sub"] = sub
    confounds_run["run_id"] = int(f.split('/')[-1].split('_')[2].split('-')[-1])
    col_list = confounds_run.columns.tolist()
    confounds_run = confounds_run.loc[:, col_list]
    confounds.append(confounds_run)
    
try:
    confounds = pd.concat(confounds).reset_index(drop=True)
except InvalidIndexError as e:
    print(
        "This error is likely due to the different number of *_comp_cor columns in "
        "different runs.\nIf that's the case, read each run separately or set "
        "'exclude_compcor' to True."
    )
    raise e

confounds_cur_list = [
            "trans_x",
            "trans_y",
            "trans_z",
            "rot_x",
            "rot_y",
            "rot_z",
            "framewise_displacement",
            "std_dvars",
            "dvars",
            "rmsd",
            "global_signal",
            "csf",
            "white_matter",
            "csf_wm",
            'a_comp_cor_00', 'a_comp_cor_01','a_comp_cor_02','a_comp_cor_03','a_comp_cor_04','a_comp_cor_05',
        ]
if confounds_cur_list:
    confounds = confounds.loc[:, ["sub", "run_id"] + confounds_cur_list]
    
# Make motion confounds regressor file
confounds_file = make_confounds_regressor(df = confounds,
    out_dir = Path(confound_dir),
    demean = True,
    split_into_runs = True,
    confounds_list = confounds_list)

for f in confounds_file:
    # check and remove allzero confounds column
    _, bad_col_idx = remove_allzero_column(f)
    if len(bad_col_idx) != 0:
        print(f"All zero column: {' '.join(list(np.array(f)[bad_col_idx]))}")
        
# Make good TR file
goodtr_file, _ = make_good_tr_regressor(
    confounds,
    Path(confound_dir),
    fd_thresh=fd_thresh,
    std_dvars_thresh=std_dvars_thresh,
    censor_prev_tr=False,
)

Confounds regressor: trans_x, trans_y, trans_z, rot_x, rot_y, rot_z, framewise_displacement, a_comp_cor_00, a_comp_cor_01, a_comp_cor_02, a_comp_cor_03, a_comp_cor_04, a_comp_cor_05, csf.
Mean center all regressors within each run.
Framewise Displacement threshold: 0.5
Standards DVARS threshold: 1.5
Total censored TR number: 164(12.20%)


Mean center a subset? - important for confounds, provide easier explanations for confounds

In [8]:
func_file = func_file_list[0]
func_id = 0

In [9]:
# Calculate run length
run_length = calc_run_length(Path(func_file), cifti=True)[0]
if run_length != num_TRs:
    print(f'==============={func_file} has incorrect number of TRs, please check raw files===============')
    

In [10]:
fname_prefix = '_'.join(func_file.split('/')[-1].split('_')[:4])
scaled_file = scale_func_image(
        func_file, Path(tmp_dir, f"{fname_prefix}_scaled.nii.gz"), mask_file = mask_intersect, cifti=False
    )

++ 3dTstat: AFNI version=AFNI_24.0.17 (Mar 24 2024) [64-bit]
++ Authored by: KR Hammett & RW Cox
++ Output dataset /tmp/tmpvp9bqwd_/mean.nii.gz

++ 3dcalc: AFNI version=AFNI_24.0.17 (Mar 24 2024) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/temporary_files/sub-GLACIER03_task-glacier_run-1_space-T1w_scaled.nii.gz



In [23]:
fname_prefix

'sub-GLACIER03_task-glacier_run-1_space-T1w'

In [12]:
confounds_file[func_id]

PosixPath('/projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/confound_files/1_confounds.1D')

In [13]:
timing_file = timing_file_list[func_id]
timing = pd.read_csv(timing_file)

behav_file = behav_file_list[func_id]
behav = pd.read_csv(behav_file)

curr_confounds_file = confounds_file[func_id]
curr_goodtr_file = goodtr_file[func_id]

print(func_file)
print(timing_file)
print(behav_file)
print(curr_confounds_file)
print(curr_goodtr_file)

/projects/kuhl_lab/wanjiag/GLACIER/derivatives/fmriprep/sub-GLACIER03/func/sub-GLACIER03_task-glacier_run-1_space-T1w_desc-preproc_bold.nii.gz
/projects/kuhl_lab/wanjiag/GLACIER/derivatives/behavior/sub03/sub03_exposure0_timing_2024-01-31_14h20.02.893.csv
/projects/kuhl_lab/wanjiag/GLACIER/derivatives/behavior/sub03/sub03_exposure0_behav_2024-01-31_14h20.02.893.csv
/projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/confound_files/1_confounds.1D
/projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/confound_files/1_goodtr.1D


In [59]:
os.path.basename(curr_confounds_file).split('_')[0]

'1'

In [21]:
goodtr_id = int(re.findall(r'\d', os.path.basename(curr_goodtr_file).split('_')[1])[0])

1

In [26]:
run_id = int(re.findall(r'\d', os.path.basename(scaled_file).split('_')[2])[0])
behav_id = int(re.findall(r'\d', os.path.basename(behav_file).split('_')[1])[0]) + 1 #behav start from 0
timing_id = int(re.findall(r'\d', os.path.basename(timing_file).split('_')[1])[0]) + 1 #behav start from 0
confound_id = int(re.findall(r'\d', os.path.basename(curr_confounds_file).split('_')[1])[0]) #behav start from 0
goodtr_id


if run_id != behav_id or run_id != timing_id or run_id != confound_id or run_id != goodtr_id:
    print(f'==============={func_file} has incorrect files: index inconsistent===============')

In [27]:
design_matrix = timing.loc[(timing['stimli'] != 'fixation') & (timing['stimli'] != 'resp-6')].reset_index(drop=True)

In [28]:
design_matrix = design_matrix.drop_duplicates(subset=['stimli']).loc[:,['stimli', 'design_onset']]

In [29]:
trl_list = design_matrix["stimli"].values
time_list = design_matrix['design_onset'].values

In [30]:
trl_list

array(['gazebo10', 'gazebo20', 'beach20', 'beach2', 'beach13', 'gazebo12',
       'gazebo19', 'lure-gazebo0', 'gazebo8', 'gazebo22', 'beach5',
       'beach9', 'gazebo4', 'beach12', 'beach0', 'beach19', 'gazebo7',
       'gazebo17', 'lure-beach20', 'gazebo6', 'lure-gazebo20',
       'lure-gazebo3', 'gazebo15', 'gazebo0', 'beach7', 'gazebo3',
       'gazebo14', 'lure-beach0', 'beach11', 'beach16', 'beach18',
       'gazebo23', 'gazebo5', 'gazebo13', 'beach21', 'beach1', 'beach6',
       'beach17', 'beach3', 'beach10', 'gazebo9', 'gazebo2', 'beach15',
       'lure-beach22', 'beach23', 'gazebo21', 'gazebo18', 'beach22',
       'gazebo16', 'gazebo11', 'beach4', 'beach14', 'beach8', 'gazebo1'],
      dtype=object)

In [56]:
len(trl_list)

54

In [31]:
time_list

array([  6.,  12.,  18.,  24.,  30.,  36.,  42.,  48.,  54.,  60.,  66.,
        72.,  78.,  84.,  90.,  96., 102., 108., 114., 120., 126., 132.,
       138., 144., 150., 156., 162., 168., 174., 180., 186., 192., 198.,
       204., 210., 216., 222., 228., 234., 240., 246., 252., 258., 264.,
       270., 276., 282., 288., 294., 300., 306., 312., 318., 324.])

In [32]:
trl_id = trl_list[0]

In [33]:
print(f"\n########## Fitting model for {run_id} trial: {trl_id} ##########",
                    flush=True,)


########## Fitting model for 1 trial: gazebo10 ##########


In [34]:
# Make main regressors onset time file
onset_file, _ = make_singletrial_onset_time(
    design_matrix, trl_id, Path(xmat_dir), prefix=fname_prefix
)

In [35]:
# Make model design matrix
xmat_file = make_singletrial_design_matrix(
    onset_file,
    trl_id,
    Path(xmat_dir),
    run_length,
    repetition_time,
    confounds_file=curr_confounds_file,
    goodtr_file=curr_goodtr_file,
    stim_times_subtract=stim_times_subtract,
    prefix=fname_prefix,
)

/home/wanjiag/abin/3dDeconvolve -DAFNI_USE_ERROR_FILE=NO -nodata 168 2.0 -local_times -polort A -num_stimts 2 -stim_times 1 /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/xmat_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trlgazebo10_ev-Target.1D 'SPMG1(6)' -stim_label 1 Target -stim_times 2 /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/xmat_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trlgazebo10_ev-Other.1D 'SPMG1(6)' -stim_label 2 Other -x1D /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/xmat_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trlgazebo10_xmat.1D -x1D_uncensored /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/xmat_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trlgazebo10_nocensor_xmat.1D -x1D_stop -ortvec /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/confound_files/1_confounds.1D confounds -censor /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/confound

In [36]:
xmat_file

PosixPath('/projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/xmat_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trlgazebo10_xmat.1D')

In [37]:
trial_prefix = f"{fname_prefix}_trial-{trl_id}"

In [38]:
trial_prefix

'sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10'

In [39]:
# Fit model use AFNI's 3dREMLfit
bucket_file = fit_3dREMLfit_cifti_separate(
    xmat_file,
    bucket_dir,
    volume_file=scaled_file,
    volume_mask_file=mask_intersect,
    prefix=trial_prefix,
)

/home/wanjiag/abin/3dREMLfit -input /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/temporary_files/sub-GLACIER03_task-glacier_run-1_space-T1w_scaled.nii.gz -mask /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/temporary_files/sub-GLACIER03_space-T1w_desc-brain_intersect_mask.nii.gz -matrix /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/xmat_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trlgazebo10_xmat.1D -Rbuck /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_fitted_volume_bucket.nii.gz -tout -rout -noFDR -nobout 
++ 3dREMLfit: AFNI version=AFNI_24.0.17 (Mar 24 2024) [64-bit]
++ Authored by: RWCox
++ Number of voxels in mask = 268452
++ Loading input dataset into memory
 + masked off 193 voxels for being all zero; 268259 left in mask
++ Converting input dataset to vector image
 +  dataset = 441 million bytes
 +  vectim  = 181 million bytes
++ s

In [47]:
bucket_file

PosixPath('/projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_fitted_volume_bucket.nii.gz')

In [53]:
def extract_sub_bucket_3dTcat(
    input_bucket_file,
    out_dir,
    prefix = None,
) -> Path:
    """Extract individual volumn from 3dREMLfit outputs

    This function extracts Target#0_Tstat|Target_R^2 from outputs
    Assumed output subject bucket sequence: 
    Full_R^2|Full_Fstat|Target#0_Coef|Target#0_Tstat|Target_R^2|Other#0_Coef|Other#0_Tstat|Other_R^2
    Can be checked with 3dinfo -label command

    Args:
        input_bucket_file: Input bucket file path.
        out_dir: Directory to store output files.
        prefix: The output filename prefix (before .dscalar.nii).
            If None, use default names.

    Returns:
        A list of 2 files for tstats and R^2 volumn.

    Raises:
        ValueError: None of the input file is specified.
        ValueError: Input file's format is incorrect.
    """
    
    # Parse prefix
    prefix = "single-vol" if prefix is None else f"{prefix}_single-vol"
    out_files = [f"{out_dir}/{prefix}_target-tstat.nii.gz",
                f"{out_dir}/{prefix}_target-R2.nii.gz"]
    cmd = (
        f"{afni_prefix}3dTcat -prefix {out_files[0]} {input_bucket_file}[3]"
    )
    print(cmd)
    run_cmd(cmd, cwd=out_dir)
    
    cmd = (
        f"{afni_prefix}3dTcat -prefix {out_files[1]} {input_bucket_file}[4]"
    )
    print(cmd)
    run_cmd(cmd, cwd=out_dir)

    return out_files

In [54]:
extract_sub_bucket_3dTcat(
    input_bucket_file = bucket_file,
    out_dir = Path(bucket_dir),
    prefix = trial_prefix,
)

/home/wanjiag/abin/3dTcat -prefix /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_single-vol_target-tstat.nii.gz /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_fitted_volume_bucket.nii.gz[3]
++ 3dTcat: AFNI version=AFNI_24.0.17 (Mar 24 2024) [64-bit]
++ elapsed time = 0.4 s

/home/wanjiag/abin/3dTcat -prefix /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_single-vol_target-R2.nii.gz /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_fitted_volume_bucket.nii.gz[4]
++ 3dTcat: AFNI version=AFNI_24.0.17 (Mar 24 2024) [64-bit]
++ elapsed time = 0.3 s



['/projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_single-vol_target-tstat.nii.gz',
 '/projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_single-vol_target-R2.nii.gz']

In [49]:
!/home/wanjiag/abin/3dTcat -prefix /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/xmat_filessub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_single-vol_target-tstat.nii.gz /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_fitted_volume_bucket.nii.gz[3]

++ 3dTcat: AFNI version=AFNI_24.0.17 (Mar 24 2024) [64-bit]
** ERROR: output dataset name 'xmat_filessub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_single-vol_target-tstat.nii.gz' conflicts with existing file
** ERROR: dataset NOT written to disk!
++ elapsed time = 0.3 s


In [ ]:
3dTcat -prefix ./sub-GLACIER02_task-glacier_run-1_space-T1w_trial-gazebo7_fitted_target-tstats.nii.gz sub-GLACIER02_task-glacier_run-1_space-T1w_trial-gazebo7_fitted_volume_bucket.nii.gz[3]

In [ ]:
3dTcat -prefix ./sub-GLACIER02_task-glacier_run-1_space-T1w_trial-gazebo7_fitted_targetr2.nii.gz sub-GLACIER02_task-glacier_run-1_space-T1w_trial-gazebo7_fitted_volume_bucket.nii.gz[4]

In [43]:
!/home/wanjiag/abin/3dinfo -label /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER02/bucket_files/sub-GLACIER02_task-glacier_run-1_space-T1w_trial-gazebo7_fitted_volume_bucket.nii.gz

Full_R^2|Full_Fstat|Target#0_Coef|Target#0_Tstat|Target_R^2|Other#0_Coef|Other#0_Tstat|Other_R^2


In [44]:
!/home/wanjiag/abin/3dinfo -label /projects/kuhl_lab/wanjiag/GLACIER/derivatives/glm/sub-GLACIER03/bucket_files/sub-GLACIER03_task-glacier_run-1_space-T1w_trial-gazebo10_fitted_volume_bucket.nii.gz

Full_R^2|Full_Fstat|Target#0_Coef|Target#0_Tstat|Target_R^2|Other#0_Coef|Other#0_Tstat|Other_R^2


In [4]:
!/gpfs/projects/kuhl_lab/wanjiag/packages/workbench/bin_linux64/wb_command

Version: 1.5.0
Commit Date: 2021-02-16 13:46:47 -0600
Operating System: Linux

Information options:
   -help                       show this help info
   -arguments-help             explain the format of subcommand help info
   -global-options             display options that can be added to any command
   -parallel-help              details on how wb_command uses parallelization
   -cifti-help                 explain the cifti file format and related terms
   -gifti-help                 explain the gifti file format (metric, surface)
   -volume-help                explain volume files, including label volumes
   -version                    show extended version information
   -list-commands              list all processing subcommands
   -list-deprecated-commands   list deprecated subcommands
   -all-commands-help          show all processing subcommands and their help
                                  info - VERY LONG

To get the help information of a processing subcommand, run it wi

/home/wanjiag/projects/environments/glacier_env/bin:/packages/miniconda-t2/20230523/condabin:/packages/miniconda-t2/20230523/bin:/gpfs/t2/slurm/apps/current/sbin:/gpfs/t2/slurm/apps/current/bin:/packages/miniconda-t2/20230523/condabin:/var/www/ood/apps/sys/dashboard/tmp/node_modules/yarn/bin:/opt/ood/ondemand/root/usr/share/gems/3.1/bin:/opt/ood/ondemand/root/usr/bin:/opt/ood/ondemand/root/usr/sbin:/sbin:/bin:/usr/sbin:/usr/bin
